

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_TR_NEWS.ipynb)






# **Sentiment Analysis of German texts**

## 1. Colab Setup

In [1]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-09-22 15:14:31--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-09-22 15:14:32--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-09-22 15:14:32--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:44

In [2]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Start Spark Session

In [5]:
spark = sparknlp.start()

## 3. Select the DL model and re-run all the cells below

In [6]:
model_name = "bert_sentiment_de"

## 4. Some sample examples

In [4]:
text_list = [
"""Spiel und Meisterschaft nicht spannend genug? Muss man jetzt den Videoschiedsrichter kontrollieren? Ich bin entsetzt,dachte der darf nur bei krassen Fehlentscheidungen ran. So macht der Fussball keinen Spass mehr.""",             
"""Habe gestern am Mittwoch den werder Podcast vermisst. Wie schnell man sich an etwas gewöhnt und darauf freut. Danke an Plainsman für die guten Interviews und den Einblick hinter die Kulissen von werderbremen. Angenehme Winterpause weiterhin!""",
"""nuernberg Ich habe sie erworben ohne ein Abonnement. Daher macht der Satz nur bedingt Sinn.""",
"""ich mag den Schiri nicht, denn der ist vergiftet!"""
"""ManCity Guardiola und seine Gang, ihr seid Arschlöcher. Ich habe gerade ein Vermögen durch deine Schuld deine Bayern dort verloren""",
]

## 5. Define Spark NLP pipeline

In [7]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = BertSentenceEmbeddings\
    .pretrained('labse', 'xx') \
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

document_classifier = ClassifierDLModel.load(model_name) \
  .setInputCols(["document", "sentence_embeddings"]) \
  .setOutputCol("class")    
     
nlpPipeline = Pipeline(stages=[
 document, 
 embeddings,
 document_classifier
 ])

labse download started this may take some time.
Approximate size to download 1.7 GB
[OK!]


## 6. Run the pipeline

In [8]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)

## 7. Visualize results

In [9]:

result.select(F.explode(F.arrays_zip('document.result', 'class.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("class")).show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|document                                                                                                                                                                                                                                         |class   |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Spiel und Meisterschaft nicht spannend genug? Muss man jetzt den Videoschiedsrichter kontrollieren? Ich bin entsetzt,dachte der darf nur bei krassen Fehlentscheidungen ran. So macht der Fussball keinen Spass mehr.                           